# **🇯🇵 Kanji Handwritten Recognition - Interactive Demo (Gradio)**

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1️⃣ CÀI ĐẶT THƯ VIỆN CẦN THIẾT


In [14]:
print("STEP 1: Installing Gradio...")
!pip install -q gradio

print("✅ Gradio installed successfully.")

STEP 1: Installing Gradio...
✅ Gradio installed successfully.


# 2️⃣ THIẾT LẬP CẤU HÌNH VÀ TẢI ARTIFACTS


In [15]:
print("\nSTEP 2: Setting up Configuration and Loading Artifacts...")
import gradio as gr
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import numpy as np
import json
from pathlib import Path

# --- Cấu hình (PHẢI TRÙNG KHỚP VỚI FILE HUẤN LUYỆN) ---
WORK_DIR = Path('/content/drive/MyDrive/kanji_handwritten_recognition')
MODEL_PATH = WORK_DIR / 'models' / 'best_model.pth'
IDX2CHAR_PATH = WORK_DIR / 'idx2char.json'
IMG_SIZE = 96  # Kích thước ảnh mà mô hình nhận đầu vào
DEVICE = torch.device('cpu') # Demo sẽ chạy trên CPU

# --- Tải file ánh xạ (mapping) ---
try:
    with open(IDX2CHAR_PATH, 'r', encoding='utf-8') as f:
        idx2char = json.load(f)
    # Chuyển key từ string về integer
    idx2char = {int(k): v for k, v in idx2char.items()}
    NUM_CLASSES = len(idx2char)
    print(f"✅ Loaded character map with {NUM_CLASSES} classes.")
except FileNotFoundError:
    raise FileNotFoundError(f"Could not find idx2char.json at {IDX2CHAR_PATH}. Please ensure the training pipeline ran successfully.")


STEP 2: Setting up Configuration and Loading Artifacts...
✅ Loaded character map with 93 classes.


# 3️⃣ ĐỊNH NGHĨA LẠI KIẾN TRÚC MÔ HÌNH


In [16]:
print("\nSTEP 3: Redefining the Model Architecture...")

def build_resnet18_grayscale(num_classes: int, dropout_rate: float = 0.5):
    """
    Hàm này PHẢI GIỐNG HỆT hàm đã dùng trong quá trình huấn luyện
    để đảm bảo kiến trúc khớp với trọng số đã lưu.
    """
    model = models.resnet18(weights=None)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(p=dropout_rate),
        nn.Linear(num_features, num_classes)
    )
    return model

# --- Tải mô hình và trọng số đã huấn luyện ---
try:
    model = build_resnet18_grayscale(num_classes=NUM_CLASSES).to(DEVICE)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model.eval() # Rất quan trọng: Chuyển mô hình sang chế độ đánh giá
    print("✅ Model architecture defined and weights loaded successfully.")
except FileNotFoundError:
    raise FileNotFoundError(f"Could not find best_model.pth at {MODEL_PATH}. Please ensure the training pipeline ran successfully.")


STEP 3: Redefining the Model Architecture...
✅ Model architecture defined and weights loaded successfully.


# 4️⃣ ĐỊNH NGHĨA HÀM TIỀN XỬ LÝ VÀ DỰ ĐOÁN

In [24]:
print("\nSTEP 4: Defining Preprocessing and Prediction Functions...")

# --- Các phép biến đổi cho ảnh đầu vào (PHẢI GIỐNG HỆT eval_transforms) ---
preprocess_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

def predict(image: dict) -> dict:
    """
    Hàm nhận dictionary đầu vào từ Gradio, trích xuất ảnh,
    tiền xử lý và trả về dự đoán.
    """
    # Lấy mảng NumPy của ảnh từ dictionary
    # THAY ĐỔI CHÍNH LÀ ĐÂY
    drawing_data = image['composite']

    if drawing_data is None:
        return {}

    # Lật ngược màu sắc (chữ đen trên nền trắng -> chữ trắng trên nền đen)
    inverted_image = 255 - drawing_data

    # Chuyển mảng NumPy thành ảnh PIL và chuyển sang ảnh xám
    pil_image = Image.fromarray(inverted_image).convert('L')

    # Các bước còn lại giữ nguyên
    image_tensor = preprocess_transform(pil_image).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        logits = model(image_tensor)
        probabilities = torch.nn.functional.softmax(logits, dim=1)[0]

    top5_probs, top5_indices = torch.topk(probabilities, k=5)
    confidences = {idx2char[idx.item()]: prob.item() for idx, prob in zip(top5_indices, top5_probs)}

    return confidences

print("✅ Prediction function is ready.")


STEP 4: Defining Preprocessing and Prediction Functions...
✅ Prediction function is ready.


# 5️⃣ KHỞI TẠO VÀ CHẠY GIAO DIỆN GRADIO


In [25]:
print("\nSTEP 5: Launching the Gradio Interface...")
print("Please wait a moment for the Gradio link to appear...")

# Tạo giao diện
iface = gr.Interface(
    fn=predict,
    inputs=gr.Sketchpad(
        type="numpy",
        width=280,
        height=280,
        image_mode='RGB',
        label="Draw a Kanji Character"
    ),
    outputs=gr.Label(
        num_top_classes=5,
        label="Top 5 Predictions"
    ),
    live=True,
    title="🇯🇵 Kanji Handwriting Recognition Demo",
    description=(
        "Draw a single Kanji character in the box below using your mouse or finger. "
        "The model will predict the character in real-time. "
        "This demo uses a ResNet18 model trained on the kkanji2 dataset."
    )
)

iface.launch(debug=True, share=True)


STEP 5: Launching the Gradio Interface...
Please wait a moment for the Gradio link to appear...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://899ee56a8999c06073.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


content_type multipart/form-data; boundary=----WebKitFormBoundaryp6SYCfejgqDMBgmL
content_type multipart/form-data; boundary=----WebKitFormBoundaryeSOE2AJAjn4LEHDw
content_type multipart/form-data; boundary=----WebKitFormBoundary09BBFdpIZRcy9yIo
content_type multipart/form-data; boundary=----WebKitFormBoundaryLDy3E9MyFvP66KAD
content_type multipart/form-data; boundary=----WebKitFormBoundarybhGTPRsWPMw9GKB8
content_type multipart/form-data; boundary=----WebKitFormBoundaryPX6jNPHvljauwKrU
content_type multipart/form-data; boundary=----WebKitFormBoundary7oAy6nbf8hBMFMFA
content_type multipart/form-data; boundary=----WebKitFormBoundary4pPPKTEm67e5e4mg
content_type multipart/form-data; boundary=----WebKitFormBoundaryL6P8HCABDBl4DBYb
content_type multipart/form-data; boundary=----WebKitFormBoundaryo4dXtRiDgctKRMA4
content_type multipart/form-data; boundary=----WebKitFormBoundaryt0t750OIT8FoV5DW
content_type multipart/form-data; boundary=----WebKitFormBoundary8cvkIBCP8FwA0lTP
content_type mul